# **CHAPTER_6**

## **6.1 텍스트 파일에서 데이터를 읽고 쓰는 법**

텍스트 데이터를 데이터프레임으로 읽어오기 위해 함수를 작성하고 색인, 자료형, 반복 여부 등에 대한 옵션을 지정한다. 특히 read_csv의 경우 데이터 형식에 자료형이 포함되어있지 않으므로 타입 추론을 수행한다. 자료를 읽어올 때 칼럼명, 색인 등을 지정할 수 있으며 공백 등의 구분자나 일반적이지 않은 구분자에 대해서도 옵션을 지정하여 데이터를 읽어올 수 있다. 이는 정규 표현식을 이용하며 여러 개의 공백 문자로 구분되어 있을 경우 |s+를 이용하여 데이터를 읽어온다. 

In [ ]:
!cat /content/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [ ]:
# 쉼표로 구분되어 있는 파일에 대해 read_csv를 이용해 데이터프레임으로 읽어온다. 

import pandas as pd
pd.read_csv('/content/ex1.csv')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [ ]:
# 칼럼 명 혹은 색인을 지정하여 데이터 프레임을 반환할 수 있다. 

names = ['a', 'b', 'c', 'd', 'message']
df = pd.read_csv('/content/ex2.csv', names = names, index_col='message')
df.index

Index(['hello', 'world', 'foo'], dtype='object', name='message')

In [ ]:
# 데이터에 결측치가 포함된 경우 na_values 옵션을 사용하여 누락된 값을 처리할 수 있다. 또한 NA 값으로 채울 위치를 지정할 수 있다. 

pd.read_csv('/content/ex5.csv', na_values=['NULL'])

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


### **6.1.1 텍스트 파일 조금씩 읽어오기**

파일에서 일부 열을 읽기 위한 옵션을 지정할 수 있으며 이는 대용량 데이터에 대해 효과적인 방법이다. chunksize 옵션을 사용하면 파일을 분할하여 읽을 수 있다. 이는 반복문으로 순회하여 파일을 읽을 수 있으며 TextParser 객체를 이용해 특정 옵션을 지정할 수 있다. sys 라이브러리의 stdout 메서드를 이용하면 실제 파일에 결과를 기록하지 않고 출력된 값을 확인할 수 있다. 

In [ ]:
pd.read_csv('/content/ex6.csv', chunksize = 100)

In [ ]:
# 'key' 로우에 위치한 값 count
 
chunker = pd.read_csv('/content/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


### **6.1.2 데이터를 텍스트 형식으로 기록하기**

In [ ]:
# Series에서도 to_csv 메서드를 사용할 수 있다. 이는 date 데이터를 쓰기에 편리하다. 
import numpy as np

dates = pd.date_range('1/1/2021', periods = 7)
ts = pd.Series(np.arange(7), index = dates)
ts.to_csv('/content/tseries.csv')
!cat /content/tseries.csv

,0
2021-01-01,0
2021-01-02,1
2021-01-03,2
2021-01-04,3
2021-01-05,4
2021-01-06,5
2021-01-07,6


### **6.1.3 구분자 형식 다루기**

csv.reader 함수를 이용하면 구분자가 한 글자인 파일을 처리할 수 있으며 파일을 읽듯이 reader를 순회하면 구분자가 제거된 튜플을 얻을 수 있다. zip(*values)를 이용하면 사전 표기법과 로우를 칼럼으로 전치함으로서 데이터 컬럼 사전을 만들 수 있다. 읽어오기 위한 함수를 새로운 클래스를 정의해서 해결할 수도 있다. 

In [ ]:
import csv
f = open('/content/ex7.csv')

reader = csv.reader(f)

In [ ]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [ ]:
with open('/content/ex7.csv') as f:
  lines = list(csv.reader(f))

header, values = lines[0], lines[1:]
header

['a', 'b', 'c']

zip 내장함수를 통해 동일한 개수로 이루어진 자료형을 묶을 수 있다. 기존에는 두 리스트를 만들고 이전 리스트의 인덱스를 기준으로 딕셔너리를 생성하였다면 zip을 이용해 더욱 간략하게 딕셔너리를 만들 수 있다. zip(*iterable)에서 iterable은 반복 가능한 자료형 여러 개를 입력할 수 있음을 의미한다. 

> (참고) https://wikidocs.net/32



In [ ]:
{h: v for h, v in zip(header, zip(*values))}

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

### **6.1.4 JSON(JavaScript Object Notation) 데이터** 

XML, YAML과 함께 효율적으로 데이터를 저장하고 교환하기위해 사용하는 텍스트 데이터 포맷 중 하나이다. 웹브라우저와 다른 애플리케이션이 HTTP 요청으로 데이터를 보낼 때 널리 사용하는 표준 파일 형식이다. 기본 자료형은 객체, 배열, 문자열, 숫자, 불리언, 널로 구성되어 있으며 **객체의 키는 반드시 문자열**이여야한다. JSON 문자열을 파이썬 형태로 변환하기 위해서는 json.loads를 사용한다. 

* 이름/값 쌍의 집합 (A collection of name/value pairs): object, record, struct, dictionary, hash table, keyed list, associative array
* 정렬된 값의 리스트 (An ordered list of values): array, vector, list, sequence



In [ ]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [ ]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 30, 'name': 'Scott', 'pets': ['Zeus', 'Zuko']},
  {'age': 38, 'name': 'Katie', 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [ ]:
json.dumps(result)

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

### **6.1.5 XML과 HTML: 웹 스크래핑**

read_html 내장 함수는 라이브러리를 사용하여 자동으로 HTML 파일을 파싱하여 데이터프레임으로 변환해준다. conda를 사용하지 않는 경우 lxml을 설치하여 대체할 수 있다. read_html의 경우 table 태그 안에 있는 모든 표 형식의 데이터 파싱을 시도한다. 결과는 데이터프레임 객체 리스트에 저장된다. 

In [ ]:
tables = pd.read_html('/content/fdic_failed_bank_list.html')

In [ ]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


**lxml.objectify를 이용해서 XML 파싱하기**

XML은 게층적 구조와 메타데이터를 포함하는 중첩된 데이터 구조를 지원하는 데이터 형식이다. XML이 범용적으로 사용되며 lxml.objectify를 이용해서 파일을 파싱한 후 루트 노드에 대한 참조를 얻어올 수 있다. XML 엘리먼트를 꺼낼 수 있으며 태그 이름을 키 값으로 하는 사전을 만들 수 있다. 

In [ ]:
from lxml import objectify

path = '/content/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()
root

<Element PERFORMANCE at 0x7fd97d21e6c8>

In [ ]:
data = []
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ','DESIRED_CHANGE','DECIMAL_PLACES']
for elt in root.INDICATOR:
  el_data = {}
  for child in elt.getchildren():
    if child.tag in skip_fields:
      continue
    el_data[child.tag] = child.pyval
  data.append(el_data)

data

[{'AGENCY_NAME': 'Metro-North Railroad',
  'CATEGORY': 'Service Indicators',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\n',
  'FREQUENCY': 'M',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'INDICATOR_UNIT': '%',
  'MONTHLY_ACTUAL': 96.9,
  'MONTHLY_TARGET': 95.0,
  'PERIOD_MONTH': 1,
  'PERIOD_YEAR': 2008,
  'YTD_ACTUAL': 96.9,
  'YTD_TARGET': 95.0},
 {'AGENCY_NAME': 'Metro-North Railroad',
  'CATEGORY': 'Service Indicators',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\

In [ ]:
# 태그 또한 메타데이터를 가지고 있을 수 있으며 objectify와 getroot를 이용해 태그나 링크 이름에서 어떤 필드라도 접근할 수 있다. 

from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()
root.text

'Google'

## **6.2 이진 데이터 형식**

데이터를 효율적으로 저장하기 위해 to_pickle 메서드를 이용한 pickle 직렬화를 사용한다. 직렬화된 객체는 내장 함수로 불러올 수 있으며 판다스의 read_pickle 메서드를 이용할 수도 있다. 또한 판다스는 HDF5와 Message-Pack 두 가지 바이너리 포맷을 지원한다. 

### **6.2.1 HDF5 형식 사용하기**

대량의 배열 데이터를 저장하기 위해 고안된 파일 포맷이다. HDF는 계층적 데이터 형식을 의미하며 여러 데이터 셋을 저장하고 부가 정보를 기록할 수 있다. 또한 압축 기술을 이용하여 더플라이(실시간) 압축을 지원하여 반복되는 패턴을 효과적으로 저장할 수 있다. 판다스 외에도 PyTables나 h5py 라이브러리를 이용하여 HDF5 파일에 접근할 수 있다. HDFStore은 fixed와 table 두 가지 저장 스키마를 지원하며 fixed 스키마가 속도가 더 빠르다. 또한 table의 경우 쿼리 연산을 지원한다. 

In [ ]:
frame = pd.DataFrame({'a': np.random.randn(100)})     

In [ ]:
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store['obj1'] # 사전과 유사한 형식

,a
0,0.209169
1,-0.013460
2,-1.011210
3,-0.903906
4,-0.104759
...,...
95,1.951768
96,0.773108
97,-1.178522
98,1.337033


In [ ]:
store.put('obj2', frame, format = 'table')
store.select('obj2', where = ['index >= 10 and index <= 15'])

,a
10,0.672607
11,-1.522625
12,-0.212738
13,0.523342
14,3.270555
15,0.148891


In [ ]:
frame.to_hdf('mydata.h5', 'obj3', format = 'table')

In [ ]:
# pd.read_hdf('mydata.h5','obj3', where = ['index < 5']) 기존 코드 ValueError: The file 'mydata.h5' is already opened, but not in read-only mode (as requested). 에러 발생
hdf = frame
hdf[:5]

,a
0,0.209169
1,-0.013460
2,-1.011210
3,-0.903906
4,-0.104759


### **6.2.2 마이크로소프트 엑셀 파일에서 데이터 읽어오기**

pandas는 ExcelFile 클래서는 read_excel 함수를 사용해 데이터를 읽어올 수 있다. XLS, XLSX 파일을 읽기 위해서 xlrd, openyxl 패키지를 사용한다. 여러 시트를 읽기 위해서는 ExcelFile을 생성하는 것이 좋다. 응답 객체의 json 메서드는 JSON 내용을 파이썬 사전 형태로 변환한 객체를 반환한다.  

## **6.3 웹 API와 함께 사용하기**

데이터는 여러 웹사이트의 공개 api를 사용해서 얻을 수 있는데 가장 쉬운 방법은 requests 패키지를 사용하는 것이다. 판다스 깃허브에서 최근 30개의 이슈를 가져오기 위해서는 GET HTTP 요청을 생성하여 얻을 수 있다. 

In [ ]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [ ]:
resp.json()[0]['title']

'Backport PR #39406 on branch 1.2.x (DOC: link to correct PR)'

In [ ]:
data = resp.json()
pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])

,number,title,labels,state
0,39429,Backport PR #39406 on branch 1.2.x (DOC: link ...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
1,39428,Backport PR #39376 on branch 1.2.x (REGR: writ...,"[{'id': 1625435109, 'node_id': 'MDU6TGFiZWwxNj...",open
2,39427,REF: reuse can_hold_element for NumericIndex._...,[],open
3,39426,BUG: sort_values create an unprintable object,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,39425,REF: de-duplicate MultiIndex validation,[],open
5,39424,BUG: indexing with missing labels deprecation ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
6,39423,BUG: Assert_frame_equal always raising Asserti...,"[{'id': 32815646, 'node_id': 'MDU6TGFiZWwzMjgx...",open
7,39422,BUG: pd.DataFrame is created inconsistently fr...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
8,39421,BUG: Subset slicer on Styler failed on MultiIn...,[],open
9,39420,Read hdf returns unexpected values for categor...,"[{'id': 78527356, 'node_id': 'MDU6TGFiZWw3ODUy...",open


## **6.4 데이터베이스와 함께 사용하기**

관계형 데이터베이스에는 SQL서버, PostgreSQL, MySQL 방법을 이용하는 것이 있는데 이러한 방식들을 기반으로 SQL에서 데이터를 읽어 와서 dataframe에 저장할 수 있다. 파이썬에서는 내장 sqlite3 드라이버를 사용하여 SQLite 데이터베이스를 이용할 수 있다. 

In [ ]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [ ]:
con

In [ ]:
# executemany를 이용하여 형식을 지정하고 데이터를 직접 대입할 수 있다
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [ ]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [ ]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [ ]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
